## Implementation of Variable Elimination to approximate probability queries from a Bayesian Network.

In [ ]:
# Given an arbitrary Bayesian Network, an arbitrary query variable, and arbitrary evidence, we want to be able to calculate the probability of said variable...
